## 找出看漲/看跌關鍵字

In [1]:
import numpy as np
import pandas as pd
import jieba
import jieba.analyse

jieba.set_dictionary('data/dict.idkrsi.txt')    #預設字典改繁體
jieba.analyse.set_stop_words('data/stopword.goatwang.kang.txt') #載入暫停字

df_down = pd.read_csv('data/pre-3_day_articles_down.csv',low_memory=True)
df_up = pd.read_csv('data/pre-3_day_articles_up.csv',low_memory=True)
df_down.drop(['Unnamed: 0'],axis=1, inplace=True) 
df_up.drop(['Unnamed: 0'],axis=1, inplace=True) 
df_up.head()

,post_time,title,content,type
0,2016-05-22 08:45:00,新低量隱憂 後520時代 台股關注新政府概念股！,台股本周在新政府上台及總統就職大典收周線，儘管市場一度擔心兩岸關係變數對盤面的影響，但在就職...,漲
1,2016-05-22 09:00:00,《各報要聞》國台辦、海協會強勢回應，堅持九二共識兩岸才能互動,在評價總統蔡英文就職演說為「沒有完成的答卷」後，中國大陸國台辦及海協會昨（21）日上午雙雙發...,漲
2,2016-05-22 09:01:00,《各報要聞》新教長guts撂話，私校退場董事淨身出戶,少子化衝擊高教，大學退場迫在眉睫，教育部長潘文忠一上任就宣告，將設置50億元基金，作為私校退...,漲
3,2016-05-22 09:01:00,《各報要聞》520後兩岸交鋒，台股恐失守8千,蔡英文總統520演說後，國台辦連續二日發表強硬聲明，尤其昨日沒有「九二共識」將中斷兩岸協商機...,漲
4,2016-05-22 09:05:00,《政治》蔡第一刀，駐美代表奉令6月5日離任,蔡英文政府上任第一天，突傳駐美代表沈呂巡奉指示6月5日離任。時間之緊迫，既不符慣例，也不符法...,漲


### 把停用詞存成list比對

In [16]:
stopwords = []
with open('data/stopword.goatwang.kang.txt', 'r',encoding='utf-8') as data:
    for stopword in data:
        stopwords.append(stopword.strip('\n'))
        
stopwords[:20]

['\ufeff\ufeff,',
 '',
 '、',
 '。',
 '“',
 '”',
 '《',
 '》',
 '！',
 '，',
 '：',
 '；',
 '？',
 '人民',
 '末##末',
 '啊',
 '阿',
 '哎',
 '哎呀',
 '哎喲']

### 結巴切詞且過濾停用字

In [41]:
title = []
# rec[0]是index rec[1]是row (類似axis)
# rec[1]['title']  dataframe由上到下一一挑出來
for rec in df_up.iterrows():
    templist = str(rec[1]['title']).replace('\n','').replace(',',' ')
    templist = jieba.cut(templist, cut_all=False)
    seglist=list(templist)
    newseglist = list(filter(lambda a: a not in stopwords and a != '\n', seglist))
    title.append(' '.join(newseglist))

df_up['title_cut']=title


content = []
# rec[0]是index rec[1]是row (類似axis)
# rec[1]['content']  dataframe由上到下一一挑出來
for rec in df_up.iterrows():
    templist = str(rec[1]['content']).replace('\n','').replace(',',' ')
    templist = jieba.cut(templist, cut_all=False)
    seglist=list(templist)
    newseglist = list(filter(lambda a: a not in stopwords and a != '\n', seglist))
    content.append(' '.join(newseglist))
    

df_up['content_cut']=content
df_up.head()

,post_time,title,content,type,title_cut,content_cut
0,2016-05-22 08:45:00,新低量隱憂 後520時代 台股關注新政府概念股！,台股本周在新政府上台及總統就職大典收周線，儘管市場一度擔心兩岸關係變數對盤面的影響，但在就職...,漲,新 低量 隱憂 520 時代 台股 關注 新 政府 概念股,台股 本周 新 政府 上台 總統 就職 大典 收周線 市場 一度 擔心 兩岸 關係 變數 盤...
1,2016-05-22 09:00:00,《各報要聞》國台辦、海協會強勢回應，堅持九二共識兩岸才能互動,在評價總統蔡英文就職演說為「沒有完成的答卷」後，中國大陸國台辦及海協會昨（21）日上午雙雙發...,漲,各報 要聞 國台辦 海協會 強勢 回應 堅持 九二 共識 兩岸 互動,評價 總統 蔡英文 就職 演說 完成 答卷 中國 大陸 國台辦 海協會 昨 21 上午 雙雙...
2,2016-05-22 09:01:00,《各報要聞》新教長guts撂話，私校退場董事淨身出戶,少子化衝擊高教，大學退場迫在眉睫，教育部長潘文忠一上任就宣告，將設置50億元基金，作為私校退...,漲,各報 要聞 新 教長 guts 撂話 私校 退場 董事 淨身 戶,少子化 衝擊 高教 大學 退場 迫在眉睫 教育部長 潘文忠 上任 宣告 設置 50 億元 基...
3,2016-05-22 09:01:00,《各報要聞》520後兩岸交鋒，台股恐失守8千,蔡英文總統520演說後，國台辦連續二日發表強硬聲明，尤其昨日沒有「九二共識」將中斷兩岸協商機...,漲,各報 要聞 520 兩岸 交鋒 台股 恐 失守,蔡英文 總統 520 演說 國台辦 連續 二日 發表 強硬 聲明 尤其 昨日 九二 共識 中...
4,2016-05-22 09:05:00,《政治》蔡第一刀，駐美代表奉令6月5日離任,蔡英文政府上任第一天，突傳駐美代表沈呂巡奉指示6月5日離任。時間之緊迫，既不符慣例，也不符法...,漲,政治 蔡第一刀 駐美 代表 奉令 離任,蔡英文 政府 上任 第一天 突傳 駐美 代表 沈呂巡 奉 指示 離任 時間 緊迫 不符 慣例...


### 針對標題做 tfidf , 挑出分數大於 0.5 的字詞

In [146]:
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

title_corpus = list(df_up['title_cut'])
title_corpus

vectorizer = CountVectorizer()
frequency = vectorizer.fit_transform(title_corpus)

transformer = TfidfTransformer()
tfidf = transformer.fit_transform(frequency)
word = vectorizer.get_feature_names() # 詞的矩陣

weight = tfidf.toarray()#將tf-idf矩陣抽取出来，元素a[i][j]表示j詞在i個文件中的tf-idf權重

title_df = pd.DataFrame(weight, columns=word)

In [107]:
dftest = title_df.loc[0,:] ##測試

In [137]:
##測試
mapp = dict(zip((dftest[dftest.values>0].sort_values(ascending=False)).index,
(dftest[dftest.values>0].sort_values(ascending=False)).values))

mapp

{'隱憂': 0.39167461837587264,
 '時代': 0.39167461837587264,
 '低量': 0.39167461837587264,
 '520': 0.3762830964191371,
 '關注': 0.36434450448573713,
 '概念股': 0.36434450448573713,
 '政府': 0.32725985072919855,
 '台股': 0.159972389843866}

In [144]:
title_dic = {}
for index,row in title_df.iterrows():
    rows = title_df.loc[index,:] #每一列
    key = (row[row.values>0.5].sort_values(ascending=False)).index # 把權重大於0.5的詞挑出
    value = (row[row.values>0.5].sort_values(ascending=False)).values
    mapping = dict(zip(key,value)) 
    title_dic.update(mapping) #更新
    
title_dic

{'交鋒': 0.5071268804511541,
 '起漲': 0.5296676060186716,
 '汽油': 0.5296676060186716,
 '首富': 0.5065349201020216,
 '迪士尼': 0.5065349201020216,
 '上海': 0.5065349201020216,
 '走弱': 0.5307035907509816,
 '弱勢': 0.6211536880236542,
 '表現': 0.5831527004979735,
 '歐股': 0.5119626900692265,
 '持平': 0.8050371670290349,
 '美股': 0.593224375512222,
 '價漲': 0.759007272256197,
 '金屬': 0.6510821458634903,
 '新壽': 0.5271564653102623,
 '注意': 0.5167989298999273,
 'otc': 0.5167989298999273,
 '千有': 0.5166690390416061,
 '48': 0.6284832178454391,
 '開盤': 0.5600357906232368,
 '14': 0.5165351374711022,
 '銷售': 0.5735478908869817,
 '平均漲': 0.5814185161298334,
 '146': 0.5814185161298334,
 '獨撐': 0.5194216711691199,
 '孤掌難鳴': 0.5994292876588277,
 '訴求': 0.5161980687373303,
 '出口': 0.6064513319384047,
 '台指期': 0.6139182288498861,
 '股期指': 0.5956193454864539,
 '跌幅': 0.5404938914344076,
 '豪雨': 0.7116673460977075,
 '大秀': 0.5017600008605679,
 '計畫': 0.5354590676043601,
 '龍捲風': 0.6035519503693818,
 '徐照': 0.6035519503693818,
 '出包': 0.523974049949

In [145]:
len(title_dic)

802

### 針對內文做 tfidf , 挑出分數大於 0.5 的字詞

In [155]:
content_corpus = list(df_up['content_cut'])

vectorizer = CountVectorizer()
frequency = vectorizer.fit_transform(content_corpus)

transformer = TfidfTransformer()
tfidf_content = transformer.fit_transform(frequency)
word_content = vectorizer.get_feature_names() # 詞的矩陣

weight_content = tfidf_content.toarray()#將tf-idf矩陣抽取出来，元素a[i][j]表示j詞在i個文件中的tf-idf權重

content_df = pd.DataFrame(weight_content, columns=word_content)

In [157]:
content_dic = {}
for index,row in content_df.iterrows():
    rows = content_df.loc[index,:] #每一列
    key = (row[row.values>0.5].sort_values(ascending=False)).index # 把權重大於0.5的詞挑出
    value = (row[row.values>0.5].sort_values(ascending=False)).values
    mapping = dict(zip(key,value)) 
    content_dic.update(mapping) #更新
    
content_dic

{'林奏延': 0.5380472327973566,
 '老屋': 0.5946948307475362,
 '健檢': 0.5413426960449547,
 '公升': 0.5032339401716834,
 '華為': 0.5045768916542505,
 '迪士尼': 0.5994761249484716,
 '大巨蛋案': 0.5271096469574661,
 '興雅': 0.5014243928386621,
 '黃志芳': 0.5336940543701555,
 'vestel': 0.5050906666862754,
 'nan': 1.0,
 '台達電': 0.5938512308765126,
 '研華': 0.6024835679391104,
 '營業日': 0.5973106190889131,
 '照明': 0.8519914132590454,
 '軍人': 0.5804528710202929,
 '漁民': 0.5451709577354754,
 '樂陞': 0.6307479061304094,
 '人民幣': 0.6405136788440039,
 '發電': 0.5433397634232455,
 'gap': 0.6996592695504715,
 '群聯': 0.579281917483781,
 '博通': 0.6678716376596775,
 '兩岸': 0.5317153823846651,
 '原油': 0.7338223668091047,
 '田弘茂': 0.502246322246972,
 '綠色': 0.5087268032306596,
 '泰國': 0.5193517762939045,
 '好感': 0.5208246044647312,
 '環評': 0.554441688302493,
 '超微': 0.6149590538742443,
 '格羅方德': 0.5408181889074113,
 '文化': 0.7446484607142334,
 '吳敦義': 0.5726726270305692,
 '印度': 0.5196742272257323,
 '流通': 0.5650904421268675,
 'note': 0.5228759678321027,

In [158]:
len(content_dic)

318